# 0. Importing libraries

In [1]:
import os
import sys
sys.path.append("../")
import pypsa
import pandas as pd
from scripts._helpers import get_solved_network_path, load_network

C:\Users\Yerbol\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


# 1. Define constants

In [2]:
PREFIX_TO_REMOVE = [
    "residential ",
    "services ",
    "urban ",
    "rural ",
    "central ",
    "decentral ",
]

RENAME_IF_CONTAINS = [
    "solid biomass CHP",
    "gas CHP",
    "gas boiler",
    "biogas",
    "solar thermal",
    "air heat pump",
    "ground heat pump",
    "resistive heater",
    "Fischer-Tropsch",
]

RENAME_IF_CONTAINS_DICT = {
    "water tanks": "TES",
    "retrofitting": "building retrofitting",
    # "H2 Electrolysis": "hydrogen storage",
    # "H2 Fuel Cell": "hydrogen storage",
    # "H2 pipeline": "hydrogen storage",
    "battery": "Battery storage",
    # "CC": "CC"
}

RENAME = {
    "Solar": "Solar PV",
    "solar": "Solar PV",
    "Sabatier": "methanation",
    "helmeth" : "methanation",
    "Offshore Wind (AC)": "Offshore wind",
    "Offshore Wind (DC)": "Offshore wind",
    "Onshore Wind": "Onshore wind",
    "offwind-ac": "Offshore wind",
    "offwind-dc": "Offshore wind",
    "Run of River": "Hydroelectricity",
    "Run of river": "Hydroelectricity",
    "Reservoir & Dam": "Hydroelectricity",
    "Pumped Hydro Storage": "Hydroelectricity",
    "PHS": "Hydroelectricity",
    "NH3": "ammonia",
    "co2 Store": "DAC",
    "co2 stored": "CO2 sequestration",
    "AC": "Transmission lines",
    "DC": "Transmission lines",
    "B2B": "Transmission lines",
    "solid biomass for industry": "solid biomass",
    "solid biomass for industry CC": "solid biomass",
    "electricity distribution grid": "distribution lines",
    "Open-Cycle Gas":"OCGT",
    "Combined-Cycle Gas":"CCGT",
    "gas": "gas storage",
    'gas pipeline new': 'gas pipeline',
    "gas for industry CC": "gas for industry",
    "SMR CC": "SMR",
    "process emissions CC": "process emissions",
    "Battery Storage": "Battery storage",
    'H2 Store': "H2 storage",
    'Hydrogen Storage': "H2 storage",
    'H2 fuel cell': "H2 storage",
    'H2 electrolysis': "H2 storage",
    'co2 sequestered': "CO2 sequestration",
    "solid biomass transport": "solid biomass",
    "uranium": "nuclear",
    "load": "Load shedding",
    "Lignite": "Coal"
}

PREFERRED_ORDER = pd.Index(
    [
        "uranium",
        "nuclear",
        "solid biomass",
        "biogas",
        "gas for industry",
        "coal for industry",
        "methanol",
        "oil",
        "lignite",
        "coal",
        "shipping oil",
        "shipping methanol",
        "naphtha for industry",
        "land transport oil",
        "kerosene for aviation",
        
        "transmission lines",
        "distribution lines",
        "gas pipeline",
        "H2 pipeline",
        
        "H2 Electrolysis",
        "H2 Fuel Cell",
        "DAC",
        "Fischer-Tropsch",
        "methanation",
        "BEV charger",
        "V2G",
        "SMR",
        "methanolisation",
        
        "battery storage",
        "gas storage",
        "H2 storage",
        "TES",
        
        "hydroelectricity",
        "OCGT",
        "CCGT",
        "onshore wind",
        "offshore wind",
        "solar PV",
        "solar thermal",
        "solar rooftop",

        "co2",
        "CO2 sequestration",
        "process emissions",

        "gas CHP",
        "solid biomass CHP",
        "resistive heater",
        "air heat pump",
        "ground heat pump",
        "gas boiler",
        "biomass boiler",
        "WWHRS",
        "building retrofitting",
        "WWHRS",
     ]
)

# 2. Define functions

In [3]:
def compute_costs(n, nice_name, cost_type):
    assert cost_type in ["Operational", "Capital"], "Type variable must be 'Operational' or 'Capital'"
    costs = n.statistics()[[f"{cost_type} Expenditure"]]
    new_index = [':'.join(idx) for idx in costs.index]
    costs.index = new_index
    costs.columns = [nice_name]
    return costs

def sum_costs(cap_cost_df, op_cost_df):
    total_cost = cap_cost_df + op_cost_df
    new_index = [x.split(":")[1] for x in total_cost.index]
    total_cost.index = new_index
    return total_cost

def get_total_costs(network, scenario_name):
    cap_costs = compute_costs(network, scenario_name, "Capital")
    op_costs = compute_costs(network, scenario_name, "Operational")
    total_costs = sum_costs(cap_costs, op_costs)
    
    df = total_costs.groupby(total_costs.index).sum()
    
    # convert to billions
    df = df / 1e9
    df = df.groupby(df.index.map(rename_techs)).sum()
    df.drop("-", inplace=True)
    return df

def get_investment_costs(network, scenario_name):
    cap_costs = compute_costs(network, scenario_name, "Capital")
    new_index = [x.split(":")[1] for x in cap_costs.index]
    cap_costs.index = new_index
    
    df = cap_costs.groupby(cap_costs.index).sum()
    
    # convert to billions
    df = df / 1e9
    df = df.groupby(df.index.map(rename_techs)).sum()
    df.drop("-", inplace=True)
    return df

def rename_techs(label):

    for ptr in PREFIX_TO_REMOVE:
        if label[: len(ptr)] == ptr:
            label = label[len(ptr) :]

    for rif in RENAME_IF_CONTAINS:
        if rif in label:
            label = rif

    for old, new in RENAME_IF_CONTAINS_DICT.items():
        if old in label:
            label = new

    for old, new in RENAME.items():
        if old == label:
            label = new
    return label

def get_average_electricity_price(network):
    # get objective cost in EUR
    costs = network.objective
    # get total load
    load = network.loads_t.p_set.sum().sum()
    # get costs EUR/MWh
    prices = costs / load
    return prices.round(2)

def get_generation_mix(n):
    # definde elec buses
    elec = ["AC", "low voltage"]
    elec = n.buses.query("carrier in @elec").index
    
    # elec mix from generators
    gens = n.generators.query("bus in @elec").index
    elec_mix = n.generators_t.p[gens].multiply(n.snapshot_weightings.objective,axis=0).T.groupby(n.generators.carrier).sum().T.sum()
    elec_mix["load"] /= 1e3
    
    # elec mix storage units
    elec_mix_hydro = n.storage_units_t.p.multiply(n.snapshot_weightings.objective,axis=0).T.groupby(n.storage_units.carrier).sum().T.sum()
    
    # elec mix from stores (csp)
    discharger_techs = ["csp"]
    discharger_links = n.links.query("carrier in @discharger_techs").index
    elec_mix_links = -n.links_t.p1[discharger_links].multiply(n.snapshot_weightings.objective,axis=0).T.groupby(n.links.carrier).sum().T.sum()
    
    # concatenate generaions
    total_mix = pd.concat([elec_mix, elec_mix_hydro, elec_mix_links], axis=0)
    total_mix.rename(index={"offwind-ac":"offwind", "offwind-dc":"offwind",
                            "load":"load shedding", "ror":"hydro", "lignite":"coal"}, inplace=True)
    # get generation mix in TWh
    total_mix = total_mix.groupby(total_mix.index).sum() / 1e6
    # clip negative generation
    total_mix = total_mix.clip(lower=0)
    return total_mix

def get_total_load(n):
    # definde elec buses
    elec = ["AC", "low voltage"]
    elec = n.buses.query("carrier in @elec").index
    loads = n.loads.query("bus in @elec").index
    # get total demand in TWh
    demand = n.loads_t.p_set[loads].multiply(n.snapshot_weightings.objective,axis=0).sum().sum() / 1e6
    return demand

def get_installed_capacities(n):
    gen_capacities = n.generators.groupby("carrier").p_nom.sum()
    storage_capacities = n.storage_units.groupby("carrier").p_nom.sum()
    capacities = (pd.concat([gen_capacities, storage_capacities], axis=0) / 1e3).round(4)
    if "load" in n.generators.carrier.unique():
        capacities.drop("load", inplace=True)
    return capacities

def get_optimal_capacities(n):
    gen_capacities = n.generators.groupby("carrier").p_nom_opt.sum()
    storage_capacities = n.storage_units.groupby("carrier").p_nom_opt.sum()
    capacities = (pd.concat([gen_capacities, storage_capacities], axis=0) / 1e3).round(4)
    if "load" in n.generators.carrier.unique():
        capacities.drop("load", inplace=True)
    return capacities

def get_capacity_expansion(optimal_capacity, installed_capacity):
    capacity_expansion = optimal_capacity - installed_capacity
    return capacity_expansion

In [4]:
# get the base working directory
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), "../"))
PYPSA_RESULTS_DIR = BASE_PATH + "/pypsa_data/results"

# country and horizon of interest
country_code = "AU"
horizon = "2050"

# solved network path
network = load_network(get_solved_network_path(country_code, horizon, PYPSA_RESULTS_DIR))

INFO:pypsa.io:Imported network elec_s_50flex_ec_lc1.0_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:root:Loading E:\Python\PyPSA\ji-gis-validation/pypsa_data/results\AU_2050/networks\elec_s_50flex_ec_lc1.0_Co2L-1H.nc


In [5]:
# scenario name
scenario_name = f"{country_code}_{horizon}"

# total costs by technologies
total_costs_by_techs = get_total_costs(network, scenario_name)

# investment costs by technology
investment_costs_by_techs = get_investment_costs(network, scenario_name)

# electricity price in EUR/MWh
electricity_prices = get_average_electricity_price(network)

# generation mix and demand
generation_mix = get_generation_mix(network)
total_load = get_total_load(network)

# get installed capacity
installed_capacity = get_installed_capacities(network)

# get optimal capacity
optimal_capacity = get_optimal_capacities(network)

# get capacity expansion
capacity_expansion = get_capacity_expansion(optimal_capacity, installed_capacity)

In [6]:
total_costs_by_techs

,AU_2050
Battery storage,8.038699
Biomass,0.168567
CCGT,2.158781
Coal,9.653393
Csp,0.000123
H2 storage,0.000815
Hydroelectricity,1.336564
Load shedding,2.121831
OCGT,0.017378
Offshore wind,0.000191


In [7]:
investment_costs_by_techs

,AU_2050
Battery storage,8.038699
Biomass,0.163488
CCGT,2.158781
Coal,9.653393
Csp,0.000123
H2 storage,0.000815
Hydroelectricity,1.336441
Load shedding,0.000000
OCGT,0.017378
Offshore wind,0.000191


In [8]:
electricity_prices

49.84

In [9]:
generation_mix

carrier
CCGT             0.000000e+00
OCGT             0.000000e+00
PHS              0.000000e+00
biomass          3.207359e-01
coal             0.000000e+00
csp              1.643564e-07
hydro            1.216783e+01
load shedding    2.101756e+00
offwind          1.304275e-03
oil              0.000000e+00
onwind           2.526211e+01
solar            5.361990e+02
dtype: float64

In [10]:
total_load

554.2284205763531

In [11]:
installed_capacity

carrier
CCGT          21.5535
OCGT           0.3840
biomass        0.5557
coal          22.1359
csp            0.0000
lignite        6.4915
offwind-ac     0.0000
offwind-dc     0.0000
oil            1.8064
onwind         8.6030
ror            1.4554
solar         17.3419
PHS            4.8600
hydro          4.2592
Name: p_nom, dtype: float64

In [12]:
optimal_capacity

carrier
CCGT           21.5535
OCGT            0.3840
biomass         0.5557
coal           22.1359
csp             0.0000
lignite         6.4915
offwind-ac      0.0004
offwind-dc      0.0005
oil             1.8064
onwind         34.9022
ror             1.4554
solar         515.4154
PHS             4.8600
hydro           4.2592
Name: p_nom_opt, dtype: float64

In [13]:
capacity_expansion

carrier
CCGT            0.0000
OCGT            0.0000
biomass         0.0000
coal            0.0000
csp             0.0000
lignite         0.0000
offwind-ac      0.0004
offwind-dc      0.0005
oil             0.0000
onwind         26.2992
ror             0.0000
solar         498.0735
PHS             0.0000
hydro           0.0000
dtype: float64